## EE361M Introduction to Data Mining - Assignment #5
## Rohan Nagar, Wenyang Fu, Zuhair Parvez

# Question 1 (1+1+5+3+3+2=15pts) - Logistic Regression

In this question we will be predicting whether someone will have an affair! Yes - there is data on this. See below on how to import the data.
1. Convert naffairs to a binary variable hadAffair which is 1 if had an affair and zero otherwise
2. Split the data into training and test. Use 42 as random seed and use 1/3rd of the data for testing. Our y variable is hadAffair and X matrix includes all the other variables except naffairs.
3. Train a logistic regression with almost no regularization (pass l2 (ridge) to penalty and 1,000 to the C parameter which is the inverse of regularization strength lambda. This essentially does l2 regularization but applies very little weight to the penalty term) and report the confusion matrix on the test data. Also report the accuracy for the "no affairs" class, the affairs class, and the average per-class accuracy on the test data. Average per-class accuracy is described in this [post](http://blog.dato.com/how-to-evaluate-machine-learning-models-part-2a-classification-metrics).
4. Repeat step 3 except use l2 penalty with Cs of [.001, .01,0.1, 1]. You will want to use k-fold cross validation to select the best parameter. To evaluate which parameter is best, maximize the average per-class accuracy. To help with this task, check out [GridSearchCV](http://scikit-learn.org/stable/modules/generated/sklearn.grid_search.GridSearchCV.html) and how to make your own [custom scorer](http://scikit-learn.org/stable/modules/model_evaluation.html).
5. Repeat question 4 except use l1 (i.e. Lasso) instead of l2 as the penalty type.
6. Which model produces the best average per-class accuracy? Why do you think this is the case? How do the models handle the different classes, and why is this so?

In [1]:
from pydataset import data
df = data('affairs')

# Answers

### Question 1.1

### Question 1.2

### Question 1.3

### Question 1.4

### Question 1.5

### Question 1.6

# Question 2 (2+3+2+3=10pts) - Support Vector Classifier

This question will continue to use the data from question 1 - including the training and test split data.
1. Fit a support vector classifier using the standard options on [sklearn's SVC](http://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html#sklearn.svm.SVC). Report the confusion matrix on the test data. Also report the accuracy for the no affairs class, the affairs class, and the average per-class accuracy (same as question 1).
2. Repeat question 1 except use grid search to select the best value of C within this set: [0.001, 0.01, 0.1, 1,5,10,100] and try both a radial and polynomial kernel (thus trying 14 combinations). Choose the combination that maximizes the average per-class accuracy. Use 5 folds. Report the best model, the confusion matrix, the accuracy for the no affairs class, the affairs class, and the average per-class accuracy.
3. Briefly discuss the effect of different  C,  kernel combinations.
4. Discuss your results from parts 1 and 2 and mention how they differ from Question 1's results.

# Answers

### Question 2.1

### Question 2.2

### Question 2.3

### Question 2.4

# Question 3 (2+1+3+1+3=10pts) - Regression Trees

This question is very similar to homework 4 question 1. Except now we will be using regression trees and not classification trees, and you will be addressing a regression problem (i.e., the independent variable "price" will not be binarized).

For this question, we will be using the housing dataset (see code below). 

1. Convert driveway, recroom, fullbase, gashw, airco, and prefarea to numeric dummy variables (1 for yes, zero for no)
2. Split the data into training and testing with a random seed of 42 and keeping 1/3rd of the data for testing
2. Fit a [decision tree regressor](http://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeRegressor.html#sklearn.tree.DecisionTreeRegressor) to predict price using all the data (your dummy variables plus bedrooms and bathrooms).
5. Report the root MSE on the test data.
6. How does the tree decide on a splitting point?

In [2]:
df = data('Housing')

# Answers

### Question 3.1

### Question 3.2

### Question 3.3

### Question 3.4

### Question 3.5

# Question 4 (2+5+3=10pts) - Support Vector Regression

This question will continue to use the data from question 4 - including the training and test split data.

1. Fit a support vector regression using the standard options on [sklearn's SVR](http://scikit-learn.org/stable/modules/generated/sklearn.svm.SVR.html). Report the root MSE.
2. Repeat question 1 except use grid search to select the best value of C within this set: [0.001, 0.01, 0.1, 1,5,10,100] and try both a radial and polynomial kernel (thus trying 14 combinations). Choose the combination that minimizes MSE. Use 5 folds. Report the best model and the test root MSE.
4. Discuss your results from parts 1 and 2 and how they differ from Question 4 results.

# Answers

### Question 4.1

### Question 4.2

### Question 4.3

# Question 5 (3+2+2+2+2+2+2=15pts) - Random Forest

This question will also continue to use the data from Question 1.
1. Fit a random forest model grid searching over the following values: {'n_estimators':[10, 100, 1000], 'max_features': ['auto', 'sqrt', 'log2']}. Choose the combination that maximizes the average per-class accuracy. Use 5 folds. Report the best model, the confusion matrix, the accuracy for the no affairs class, the affairs class, and the average per-class accuracy.
2. What do the n_estimators and max_features parameters do?
3. Report the features in order of importance based on the model used in part 1
4. Repeat question 1 using an AdaBoostClassifier and grid search over the following values: {'n_estimators':[50, 500, 5000], 'learning_rate': [.001, .01, .1]}
5. What does the learning_rate parameter do?
6. Report the features in order of importance based on the model used in part 4
7. Compare the results in part 1 and 4 and questions 2 and 3

# Answers

### Question 5.1

### Question 5.2

### Question 5.3

### Question 5.4

### Question 5.5

### Question 5.6

### Question 5.7